In [1]:
# Notes/comments
# Current bottleneck of the system is the paragraph ranker. It doesn't do a great job 
# for some question <-> paragraph combos. This is a potential area for improvment.

# ES does a pretty decent job and retrieving relevant documents. Sometimes it doesn't work right of
# the bat but if we switch the "searching technique" (either title or text) it usually does a decent job.

In [2]:
from document_reader import DocumentReader
from paragraph_ranker import ParagraphRanker
from document_retriever import DocumentRetriever

In [3]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
#question = "When was Michael Jordan born?" # got it right w/ title=True and Title=False
#question = "what is population of New York city?" # got it right w/ Title=True and Title=False
#question = "Who was the first president of United States?" # got it right w/ Title=True and Title=False
#question = "How tall is Eiffel Tower in Paris?" # got it right w/ title=True
#question = "How many people live in Columbus Ohio?" # failed to answer with both Title=True and Title=False
#question = "What is the population of Columbus Ohio?" # worked with Title=True
# question = "When was Novak Djokovic born?" # worked with Title=True
# question = "When did World War II start?" # worked with Title=True
# question = "When did French revolution begin?" # worked with title=False
question = "When did French revolution start?" # worked with Title=False
es_index = "wikipedia-full"
es_config = {'host':'localhost', 'port':9200}
model_name = "ktrapeznikov/albert-xlarge-v2-squad-v2"
tokenizer_name = "ktrapeznikov/albert-xlarge-v2-squad-v2"

In [5]:
dr = DocumentRetriever(es_config)

In [6]:
documents = dr.retrieve_docs(question, es_index, size=5, title=False)

In [7]:
documents

{'took': 46,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 19.79069,
  'hits': [{'_index': 'wikipedia-full',
    '_type': '_doc',
    '_id': 'A1sXGHQBPcTp58A51Jgr',
    '_score': 19.79069,
    '_source': {'document_title': 'Zuka',
     'document_text': 'ZukaZuka\n\nZenaida Gourievna Booyakovitch (1 January 1924 – 18 December 2016), known as Zuka, was an American artist of Russian descent who lived and worked in Paris. She was awarded "Chevalier" of "L’ordre des Arts et des Lettres" of France in 1990.\n\nZuka was born in 1924 in San Francisco to a family of Russian immigrants who came to the United States in 1920s. Her father was an officer in White Russian Army. Zuka received a bachelor\'s degree in Fine Arts from the University of Southern California.\n\nIn 1948 she went to Paris using the money she received from California gallery grant. Two years later, in 1950, she ma

In [8]:
pr = ParagraphRanker(documents)

In [9]:
contexts = pr.rank_paragaraphs(question, n_paragraphs=8)

/home/novarac23/Projects/jarvis/q_a_model/paragraph_ranker.py:37: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score.append(doc_1.similarity(doc_2))


In [10]:
contexts

['La patrie en danger (French: "The country (fatherland) in danger") was the start of a declaration by the French Assembly on 11 July 1792 in response to Prussia joining Austria against France. Along with the Levée en masse declared the next year, it was part of the growing idea of "people\'s war" which developed during the French Revolution, where ideology "not only conscripted manpower for the regular armies, but also inspired ordinary people to fight on their own account."',
 'Samuel Taylor Coleridge was also profoundly affected by the French revolution. Unlike Wordsworth, Coleridge was more open and receptive to the social and political world around him. He was a very versatile man and he led a life that covered many fields and his work displayed this. He was a poet of nature, romance, and the Revolution. He was a philosopher, a historian, and a political figure. The French Revolution played a great role in shaping Coleridge into each of these things.',
 'At the age of eighteen he 

In [11]:
doc_reader = DocumentReader(model_name, tokenizer_name)

All PyTorch model weights were used when initializing TFAlbertForQuestionAnswering.

All the weights of TFAlbertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFAlbertForQuestionAnswering for predictions without further training.


In [12]:
results = []

for i, context in enumerate(contexts):        
    try:
        if len(context) > 512:
            print('context length for item {i} is bigger then 512, need to perform chunking')
            chunks = list(get_chunks(context, 511))
            for chunk in chunks:
                final_answer = doc_reader.get_answer(question, chunk)
                results.append(final_answer)
        
        
        final_answer = doc_reader.get_answer(question, context)
        
        results.append(final_answer)
    except Exception as e:
        print(f'We could not process item under {i}. Reason is: {e}')

context length for item {i} is bigger then 512, need to perform chunking
context length for item {i} is bigger then 512, need to perform chunking
context length for item {i} is bigger then 512, need to perform chunking
context length for item {i} is bigger then 512, need to perform chunking


In [13]:
results

[('', 0.9999689),
 ('', 1.0),
 ('at the end of the decade', 0.47464043),
 ('', 1.0),
 ('', 1.0),
 ('', 1.0),
 ('', 1.0),
 ('', 0.9999989),
 ('close of the last century', 0.56139755),
 ('close of the last century', 0.58253634),
 ('', 0.9999589),
 ('', 0.99999964),
 ('', 0.9999305),
 ('1789', 0.834982),
 ('', 1.0),
 ('1789', 0.79961985)]